# 4장 : 머신러닝의 기본요소
---

### 구성
1. 머신러닝의 네가지 분류
2. 머신 러닝 모델 평가
3. 데이터 전처리, 특성공학, 특성 학습
4. 과대 적합과 과소 적합
5. 보편적인 머신러닝 작업 흐름
6. 요약


### 이 장에서 다룰 핵심 내용
* 분류와 회귀 이외의 머신 러닝 형태
* 머신 러닝 모델의 올바른 평가 과정
* 딥러닝을 위한 데이터 전처리
* 특성 공학
* 과대적합 문제 해결
* 머신 러닝 문제를 다루는 일반적인 작업 흐름